In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ros_nodes.particle_filter import *
from ros_nodes.probability_funcs import *
import matplotlib.pyplot as plt

import time

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
from PIL import Image as PILImage
import cv2


from utils import *
from models.CtRNet import CtRNet


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
parser = argparse.ArgumentParser()

args = parser.parse_args("")

args.base_dir = "/home/workspace/src/ctrnet-robot-pose-estimation-ros"
args.use_gpu = True
args.trained_on_multi_gpus = True
args.keypoint_seg_model_path = os.path.join(args.base_dir,"weights/panda/panda-3cam_azure/net.pth")
args.urdf_file = os.path.join(args.base_dir,"urdfs/Panda/panda.urdf")

args.robot_name = 'Panda' # "Panda" or "Baxter_left_arm"
args.n_kp = 7
args.height = 480
args.width = 640
args.fx, args.fy, args.px, args.py = 399.6578776041667, 399.4959309895833, 319.8955891927083, 244.0602823893229
args.scale = 0.5 # scale the input image size to (320,240)

# scale the camera parameters
args.width = int(args.width * args.scale)
args.height = int(args.height * args.scale)
args.fx = args.fx * args.scale
args.fy = args.fy * args.scale
args.px = args.px * args.scale
args.py = args.py * args.scale

In [3]:
trans_to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_img(cv_img,args):
    image_pil = PILImage.fromarray(cv_img)
    width, height = image_pil.size
    new_size = (int(width*args.scale),int(height*args.scale))
    image_pil = image_pil.resize(new_size)
    image = trans_to_tensor(image_pil)
    return image

CtRNet = CtRNet(args)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading keypoint segmentation model from /home/workspace/src/ctrnet-robot-pose-estimation-ros/weights/panda/panda-3cam_azure/net.pth
Camera intrinsics: [[199.8289388    0.         159.9477946 ]
 [  0.         199.74796549 122.03014119]
 [  0.           0.           1.        ]]
Robot model: Panda


In [4]:
cv_img = cv2.imread(os.path.join(args.base_dir,"images/panda.jpg"))
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
image = preprocess_img(cv_img,args)
if args.use_gpu:
    image = image.cuda()

joint_angles = np.array([ 0.0200, -0.9641, -0.0662, -2.7979, -0.0469,  1.9289,  0.9137])

cTr, points_2d, segmentation, confidence = CtRNet.inference_single_image(image, joint_angles)
print(CtRNet.cTr_to_pose_matrix(cTr))

tensor([[[ 0.8578,  0.5140, -0.0044, -0.1976],
         [ 0.0951, -0.1672, -0.9813,  0.3963],
         [-0.5051,  0.8414, -0.1923,  1.0495],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0',
       grad_fn=<CopySlices>)


In [38]:
init_std = np.array([
                        1.0e-2, 1.0e-2, 1.0e-2, # ori
                        1.0e-3, 1.0e-3, 1.0e-3, # pos
                    ])
pred_std = np.array([1.0e-4, 1.0e-4, 1.0e-4,
                     2.5e-5, 2.5e-5, 2.5e-5])
pf = ParticleFilter(num_states=6,
                    init_distribution=sample_gaussian,
                    motion_model=additive_gaussian,
                    obs_model=point_feature_obs,
                    num_particles=200)
pf.init_filter(init_std)

pf.predict(pred_std)

cam = None
gamma = 0.15
pf.update(points_2d, CtRNet, joint_angles, cam, cTr, gamma)


[5.84517401e-16 4.69059663e-16 7.26769741e-16 6.12962313e-16
 1.26253929e-16 4.60203752e-16 6.92210954e-16 4.11866715e-16
 6.65291398e-16 1.17698333e-16 6.01358349e-16 7.77122305e-16
 6.69272347e-16 7.71446680e-16 4.53017547e-16 3.26692255e-16
 3.51610695e-16 3.17734422e-16 6.89725784e-16 6.69507308e-16
 3.11784262e-16 7.42814434e-16 5.71772411e-16 7.14357854e-16
 5.92481020e-16 6.16618019e-16 7.86819527e-16 7.47249766e-16
 4.44662661e-16 2.10221347e-16 1.66875885e-16 6.80036148e-16
 5.66396030e-16 1.17121658e-16 6.59869116e-18 2.84431907e-16
 5.81300955e-16 7.33810419e-16 6.78706467e-16 8.01288419e-16
 6.89853247e-16 5.25354770e-16 7.52855316e-16 4.27300643e-16
 9.73357063e-19 7.36948431e-16 8.61376340e-17 2.03920212e-16
 5.12625441e-16 7.38510449e-16 3.54389373e-16 6.71087340e-16
 7.89361212e-16 4.01976521e-16 7.59791123e-16 6.94291568e-16
 5.20373357e-16 5.55607144e-16 2.83056500e-16 6.54896205e-16
 4.76768218e-16 7.29973978e-16 6.63211922e-16 7.26290274e-16
 8.09347841e-17 6.795399